In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime

In [ ]:
OUTPUT_PATH = 'Output/'
INPUT_PATH = 'Dataset/'

SIGNALS = [-1,0,1]
PRICE_TYPE = ['open','close','high','low']

In [ ]:
class Trade:
    
    def __init__(self) -> None:
        self.timeStamp = None
        self.signal = None
        self.price_type = None
    
    def execute(self,timeStamp:datetime ,signal:int,price_type:str) -> object:
        
        assert signal in SIGNALS, f"Invalid Signal {signal}"
        assert price_type in PRICE_TYPE, f"Invalid Price Type {price_type}"

        self.timeStamp = timeStamp
        self.signal = signal
        self.price_type = price_type
        return self
    
    def __str__(self) -> str:
        return f"{self.timeStamp} {self.signal} {self.price_type}"
    

class PrVO:

    def __init__(self,open:float,high:float,low:float,close:float,volume:int,timeStamp:datetime) -> None:
        self.open = open
        self.high = high
        self.low = low
        self.close = close
        self.volume = volume
        self.timeStamp = timeStamp
        self.returns = (close - open)/open
        self.candle_color = 'GREEN' if self.close > self.open else 'RED'

    def __repr__(self) -> str:
        return f"Price-Volume : Open={self.open}, High={self.high},Low={self.low}, Close={self.close}, Volume={self.volume}, TimeStamp={self.timeStamp}"
    
    def __str__(self) -> str:
        return f"Price-Volume : Open={self.open}, High={self.high},Low={self.low}, Close={self.close}, Volume={self.volume}, TimeStamp={self.timeStamp}"

In [ ]:
class TradingTerminal:

    def __init__(self) -> None:
        self.time=None
        self.PrVo=dict()
        self.cash_balance=1000
        self.tradeHistory=[]

    def readData(self,filepath):
        df=pd.read_excel(INPUT_PATH+filepath)
        
        data=[]
        for index,row in df.iterrows():
            data.append(PrVO(row['open'],row['high'],row['low'],row['close'],row['volume'],row['datetime']))

        data_timeframe=filepath.split('/')[-1].split('.')[0].split('_')[1]
        self.PrVo[data_timeframe]=np.array(data)
        
    def takeTrade(self,timeStamp,signal,price_type):
        trade=Trade().execute(timeStamp,signal,price_type)
        self.tradeHistory.append(trade)

    def outputTradeHistory(self):
        file=open(OUTPUT_PATH+'trade_history.txt','w')
        for trade in self.tradeHistory:
            file.write(str(trade)+'\n')

    def setTime(self,time):
        self.time=time

    def getSimpleMovingAverage(PrVo,lag,price_type):
        assert price_type in PRICE_TYPE, f"Invalid Price Type {price_type}"
        return PrVo[price_type].rolling(window=lag).mean()
    

    def Strategy1(self,lag1,lag2,price_type,timeframe):
        assert price_type in PRICE_TYPE, f"Invalid Price Type {price_type}"
        assert lag1>lag2, f"Invalid Lag {lag1} {lag2}"
        assert timeframe in self.PrVo.keys(), f"Invalid Timeframe {timeframe}"

        PrVo=self.PrVo[timeframe]

        ema1=TradingTerminal.getSimpleMovingAverage(PrVo,lag1,price_type)
        ema2=TradingTerminal.getSimpleMovingAverage(PrVo,lag2,price_type)
        

        for i in range(lag1,len(PrVo)):
            if ema1[i]>ema2[i] and ema1[i-1]<ema2[i-1] and self.cash_balance>0:
                self.takeTrade(PrVo[i].timeStamp,1,price_type)
            elif ema1[i]<ema2[i] and ema1[i-1]>ema2[i-1]:
                self.takeTrade(PrVo[i].timeStamp,-1,price_type)


In [ ]:
tt = TradingTerminal()

In [ ]:
FILE='btcusdt_1h.xlsx'

tt.readData(FILE)

In [ ]:
print(tt.getSimpleMovingAverage(tt.PrVo['1h'],10,'open'))